In [2]:
import numpy as np
import pandas as pd 
import string
import re

In [3]:
with open('all_inovices.txt', mode='r', encoding='utf8', errors='ignore') as f:
    text = f.read()

In [4]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [5]:
df = pd.DataFrame(data[1:], columns=data[0])

In [6]:
df.head()

,id,text,tag,
0,001.jpeg,Invoice,O,
1,001.jpeg,no:,O,
2,001.jpeg,51109338,B-ID,
3,001.jpeg,Date,O,
4,001.jpeg,of,O,


In [7]:
whitespace =string.whitespace
punctuation = '!"#$%&\'()*+-.:;<=>?@[\\]^_`{|}~'
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removeWhitespace = text.translate(tableWhitespace)
    removePunctuation = removeWhitespace.translate(tablePunctuation)

    return str(removePunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

In [10]:
dataClean.head(10)

,id,text,tag,
0,001.jpeg,invoice,O,
1,001.jpeg,no,O,
2,001.jpeg,51109338,B-ID,
3,001.jpeg,date,O,
4,001.jpeg,of,O,
5,001.jpeg,issue,O,
6,001.jpeg,seller,O,
7,001.jpeg,"andrews,",B-SN,
8,001.jpeg,kirby,I-SN,
9,001.jpeg,and,I-SN,


In [11]:
group = dataClean.groupby(by='id')

In [12]:
groupArray = group.get_group('001.jpeg')[['text','tag']].values
content = ''
annotations = {'entities':[]}
start = 0
end = 0
for text, label in groupArray :
    text = str(text)
    stringLength = len(text)+1

    start = end
    end = start + stringLength

    if label != 'O':
        annot = (start, end-1, label)
        annotations['entities'].append(annot)

    content = content + text + ' '

In [13]:
content

'invoice no 51109338 date of issue seller andrews, kirby and valdez 58861 gonzalez prairie lake daniellefurt, in 57228 tax id 945822137 iban gb75mcrl06841367619257 items no description qty tks clearance fast dell desktop 3,00 computer pc dual core windows 10 4/8/16gb ram 2 hp t520 thin client computer 5,00 amd gx212jc 12ghz 4gb ram tested read below 300 gaming pc desktop computer 1,00 4 12core gaming computer 3,00 desktop pc tower affordable gaming pc 8gb amd vega rgb se custom build dell optiplex 9020 5,00 mt i54570 320ghz desktop computer pc 6 dell optiplex 990 mt computer 4,00 pc quad core i7 34ghz 16gb 2tb hd windows 10 pro ee dell core 2 duo desktop 5,00 computer windows xp pro 4gb 500gb summary vat 10 total 4/13/2013 um eac eac eac eac eac eac h client becker ltd 8012 stewart summit apt 455 north douglas, az 95355 tax id 942800517 net price 209,00 37,75 400,00 464,89 221,99 269,95 168,00 net worth 5 640,17 5 640,17 net worth 627,00 188,75 400,00 1 394,67 1 079,80 840,00 vat 10 10

In [14]:
content.find("andrews,")

41

In [15]:
for start, end, label in annotations['entities']:
    print(f"{label}: '{content[start:end]}'")

B-ID: '51109338'
B-SN: 'andrews,'
I-SN: 'kirby'
I-SN: 'and'
I-SN: 'valdez'
B-IBAN: 'gb75mcrl06841367619257'
B-DATE: '4/13/2013'
B-CN: 'becker'
I-CN: 'ltd'
B-TOTAL: '6'
I-TOTAL: '204,19'


In [16]:
annotations

{'entities': [(11, 19, 'B-ID'),
  (41, 49, 'B-SN'),
  (50, 55, 'I-SN'),
  (56, 59, 'I-SN'),
  (60, 66, 'I-SN'),
  (140, 162, 'B-IBAN'),
  (722, 731, 'B-DATE'),
  (768, 774, 'B-CN'),
  (775, 778, 'I-CN'),
  (1120, 1121, 'B-TOTAL'),
  (1122, 1128, 'I-TOTAL')]}